In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import gc
import os

# Reading Data

In [2]:
# Here, I separated environment folder with data folder, so I can work on smaller sample of data when developing and then resue the same code in production.

# folder where data exists
data_folder = './data'
# the environment folder 
env_folder = 'prod' 

file_path = os.path.join(data_folder, env_folder)

In [3]:
# name of cleaned input datasets
order_file_name = 'cleaned_orders.parquet'
errand_file_name = 'cleaned_errands.parquet'

In [4]:
# Reading the data from given environment as datafframes 
df_orders =  pd.read_parquet( os.path.join(file_path,  order_file_name))
df_errands = pd.read_parquet( os.path.join(file_path,  errand_file_name))

In [5]:
print(
    'There are in total {:,} orders and {:,} errands.'.format(
        len(df_orders), len(df_errands)
    )
)

There are in total 6,300,206 orders and 2,680,892 errands.


In this analysis, we are only interested to see the ones for which we have recorded journeys meaning the customer had contacted us, thus we do an innerjoin and disregards other rows.

In [6]:
# merging two datasets and only keep orders with recorded errands
df = df_orders.merge(df_errands, on='order_id')

In [7]:
# Checking number of nulls
df.isnull().sum()

order_id                              0
PNR                                   0
order_created_at                      0
booking_system                        0
Site_Country                          0
Brand                                 0
Order_Amount                          0
currency                              0
Revenue                               0
Partner                               0
Customer_Group_Type                   0
Device                                0
client_entry_type                     0
booking_system_source_type            0
Origin_Country                        0
Destination_Country                   0
Journey_Type_ID                       0
Is_Changed                            0
Is_Canceled                           0
cancel_reason                         0
change_reason                         0
number_of_PNR                         0
week_start_day                        0
currency_code                         0
currency_to_use_conversion_rate       0


As the data is huge, we delete previous data frame and free the memory. 

In [8]:
# removing dataframes from memory to improve performance
del df_orders, df_errands
gc.collect()

0

In [9]:
print(
    'There are in total {:,} records in combined dataset.'.format(
        len(df)
    )
)

There are in total 2,680,892 records in combined dataset.


In [10]:
print(
    'Here are the list of columns: ', 
    ', '.join(df.columns)
)

Here are the list of columns:  order_id, PNR, order_created_at, booking_system, Site_Country, Brand, Order_Amount, currency, Revenue, Partner, Customer_Group_Type, Device, client_entry_type, booking_system_source_type, Origin_Country, Destination_Country, Journey_Type_ID, Is_Changed, Is_Canceled, cancel_reason, change_reason, number_of_PNR, week_start_day, currency_code, currency_to_use_conversion_rate, Order_Amount_in_USD, Revenue_in_USD, number_of_booking_system, order_number, errand_id, created, errand_category, errand_type, errand_action, errand_channel, is_test_errand, errand_action_code, errand_action_name


In [11]:
# Here, I am renaming columns to have the common prefix to manage it in an easier way

selecetd_columns = [
    # grain is in errand level
    'errand_id', 'created', 'errand_category', 'errand_type',  'errand_channel',    
    # order level data
    'order_id', 'order_created_at', 'Order_Amount_in_USD', 'Revenue_in_USD',
    'booking_system', 'number_of_booking_system', 
    'Site_Country', 'Brand', 'Partner', 'booking_system_source_type', 
    'Customer_Group_Type', 'Device', 'client_entry_type',
    'Journey_Type_ID', 'number_of_PNR',
    # important labels
    'Is_Changed', 'Is_Canceled',
    # descriptive resaons 
    'change_reason', 'cancel_reason',
    # action taken
    'errand_action_code', # shall this be mapped to errand_category ?
    'errand_action_name'
]



columns_orders = {
    'order_id': 'order:id', 
    'order_created_at': 'order:timestamp', 
    'Order_Amount_in_USD': 'order:amount',  
    'Revenue_in_USD': 'order:revenue', 
    'booking_system': 'order:booking_system', 
    'number_of_booking_system': 'order:num_of_booking_system', 
    'Site_Country': 'order:country', 
    'Brand': 'order:brand', 
    'Partner': 'order:partner', 
    'booking_system_source_type': 'order:booking_system_type', 
    'Customer_Group_Type': 'order:customer_group_type', 
    'Device': 'order:device', 
    'client_entry_type': 'order:client_entry_type', 
    'Journey_Type_ID': 'order:journey_type_id', 
    'number_of_PNR': 'order:num_of_pnr', 
    #
    'Is_Changed': 'order:is_changed', 
    'Is_Canceled': 'order:is_cancelled',
    'change_reason': 'order:change_reason', 
    'cancel_reason': 'order:cancel_reason',
}


columns_errands = {
    'errand_id': 'errand:id', 
    'created': 'errand:timestamp', 
    'errand_category': 'errand:category', 
    'errand_type': 'errand:type',  
    'errand_channel': 'errand:channel',    
    'errand_action_code': 'errand:action_code', 
    'errand_action_name': 'errand:action_name'
}


In [12]:
# Limitting the columns to selected ones 
df = df[selecetd_columns]

In [13]:
# checking rows when errand_type is null
df[df['errand_type'].isna()]

,errand_id,created,errand_category,errand_type,errand_channel,order_id,order_created_at,Order_Amount_in_USD,Revenue_in_USD,booking_system,...,Device,client_entry_type,Journey_Type_ID,number_of_PNR,Is_Changed,Is_Canceled,change_reason,cancel_reason,errand_action_code,errand_action_name
25002,52969341,2024-01-09 09:02:00,23: Legal and Authority Claims Team,None,2: Mail In,4607875866,2024-01-02 09:04:04,2786.770362,9.120100,System E,...,iPhone,b2bapi,OpenJawDouble,1,0,0,Initial - Not Changed,Undefined,1,Mail In
52178,52954329,2024-01-09 01:13:52,23: Legal and Authority Claims Team,None,1: Phone In,4607915354,2024-01-03 04:13:28,1233.830698,-34.456071,System E,...,PC,metasearch,One-way,2,0,0,Initial - Not Changed,Undefined,5,5. Force Majeure
66741,52954328,2024-01-09 01:13:50,23: Legal and Authority Claims Team,None,1: Phone In,4607088462,2024-01-03 16:31:15,655.239731,16.267452,"System E,System B",...,PC,metasearch,One-way,2,0,0,Initial - Not Changed,Undefined,5,5. Force Majeure


In [14]:
# Checking total number of rows
len(df)

2680892

The following reports are only valid for orders where there is at least one errand for it. 

In [15]:
# renaimg columns of the dataframes to new names 
df = df.rename(columns=columns_orders)
df = df.rename(columns=columns_errands)

In [16]:
# save for analysis and machine learning
df.to_parquet(os.path.join(file_path, 'errands_orders_cleaned.parquet'))

In [17]:
# changing the feature value to enable filtering orders where only one booking system is used
df['order:booking_system'] = df.apply(
    lambda row: 'combined' if row['order:num_of_booking_system'] > 1 else row['order:booking_system'],
    axis=1
)

In [18]:
# checking total number of rows
len(df)

2680892

In [19]:
# checking number of nulls
df.isnull().sum()

errand:id                         0
errand:timestamp                  0
errand:category                   0
errand:type                       3
errand:channel                    0
order:id                          0
order:timestamp                   0
order:amount                      0
order:revenue                     0
order:booking_system              0
order:num_of_booking_system       0
order:country                     0
order:brand                       0
order:partner                     0
order:booking_system_type         0
order:customer_group_type         0
order:device                      0
order:client_entry_type           0
order:journey_type_id             0
order:num_of_pnr                  0
order:is_changed                  0
order:is_cancelled                0
order:change_reason               0
order:cancel_reason               0
errand:action_code             2184
errand:action_name             2184
dtype: int64

# Extract XES

In [20]:
# renaming dataframe and map case identifier, task identifier and timestamp and rename them for processs mining
df_xes = df[list(df.columns[df.columns.str.startswith('errand:')])+['order:id']].rename(columns={
    'order:id': 'case:concept:name',
    'errand:action_name': 'concept:name',
    'errand:timestamp': 'time:timestamp',
}).copy()

In [21]:
# adding create event for all orders 
df_xes_order = df[df.columns[df.columns.str.startswith('order:')]].drop_duplicates().reset_index(drop=True).copy()
df_xes_order['concept:name'] = 'create order'
df_xes_order = df_xes_order.rename(columns={
    'order:id': 'case:concept:name',
    'order:timestamp': 'time:timestamp',
})

In [22]:
# saving the transformed data for upcoming analysis (process mining)
df_xes.to_parquet(os.path.join(file_path,'xes.parquet'))